# Lecture 3.4: Multi-Arm Bandit  

### Example Scenario  

Consider the following scenario:  


* The company you work for is testing out a new version of it's website.
* After running an A/B test for an afternoon, the new version of the site appears to performing slightly better than the old version.
* After running the test over night, the new version of the site is performing better than the old version with a statistically significant p-value of 0.04.


Do you stop the test, or do you keep running it?

## Exploration vs. Exploitation  

When we are trying to determine which option, from a potential
pool, is the best option what we are faced with is an information
gathering challenge.  

* **Exploration**: Trying out different options to try and determine
the reward associated with the given approach (i.e. acquiring
more knowledge)  
* **Exploitation**: Going with the approach that you believe to
have the highest expected payoff (i.e. optimizing decisions
based on existing knowledge)

### Traditional A/B Testing  

* A short period of *pure exploration*, in which you assign
equal numbers of users to Group A and Group B  
* A long period of *pure exploitation*, in which you send
all of your users to the more successful version of your
site and never come back to the option that seemed to
be inferior  

**Q**: Why might this be a bad strategy?  

#### Potential Inefficiencies  

* Equal number of observations are routed to A and B for a
preset amount of time or iterations  
* Need to wait for the experiment to conclude for certain
statistical guarantees to be provided  
* Only after that preset amount of time or iterations do we stop
and use the better performer  
* This wastes time - and money - showing users the
suboptimal site  

## Multi-Arm Bandit Approach 

* Shows a user the site that you think is best most of the time
(exactly how is dictated by the strategy chosen)
* As the experiment runs, we update the belief about the true
CTR (Click Through Rate)
* Run for however long until we are satisfied the experiment has
determined the better site
* Balances exploration and exploitation rather than doing only
one or the other

### What Is a Multi-Armed Bandit?  

* Each slot machine (a.k.a. one-armed bandit) has a different (unknown!)
chance of winning.
* How do you maximize your total payout after a finite number of plays?
* Assume all have the same payoff. (“binary bandits”)

**Example**:  
* which version of an web ad has a higher click-through rate
(CTR)?
* which website is easier to navigate?
* which drug is more effective?

### Use Cases  

* Dynamic A/B Testing
* Budget allocation amongst competing projects
* Clinical trials
* Adaptive routing in attempts to minimize network delays
* Reinforcement learning

### Minimizing Regret  

* Regret is the difference of what we won and what we would expect with optimal strategy

$$ 
\begin{align*} 
\text{regret} &= \sum_{i = 1}^k (p_{opt} - p_i)   \\ 
&= k p_{opt} - \sum_{i = 1}^k p_i
\end{align*} $$

* where there are k trials and $p_i$ is the probability of winning with the bandit chosen on the i-th run. $p_{opt}$ is the probability of winning with the best bandit

* We can view this as a cost function we are trying to minimize  


* A regret of 0 would mean you always played the best machine - this isn't ever possible since you need to collect data to determine which one is the best

* Note that you need to know the true probabilities to calculate the regret. This is a theoretical idea to evaluate which algorithm is best and is probably not something you would calculate in practice

### Epsilon-Greedy Algorithm  

* In the epsilon-greedy algorithm, some percent of the time we explore and randomly choose one of the options. The rest of the time we choose the option that has so far had the highest conversion rate. epsilon is a value between 0 and 1 which is the probability that we explore. 10% is a standard choice for epsilon.  


* Here's the pseudocode:  

```
randomly choose a number r from 0 to 1
if r < epsilon:
    randomly choose one of the possible sites, each with equal probability
else:
    choose the site with the highest conversion rate from the data so far
```

* Each version will have seen a different number of users. You will calculate each conversion rate like this:  


```
                             # clicks from site A
conversion rate of site A = ----------------------
                              # visits of site A

```

* This algorithm is simple, but it has some flaws.

    * At the beginning, we are trying to do exploitation, but we don't yet know which version is right.

    * If epsilon is 90%, the epsilon-greedy algorithm will eventually show the best option 90% of the time. So this means that 10% of the time the algorithm will continue to randomly show different versions of the site, no matter how confident we are that a certain version is the best. There is a point where we should stop the exploration.

### Softmax  

* In this algorithm, we choose each version of the site in proportion to its estimated value using a Boltzman Distribution. Say $p_A$, $p_B$ and $p_C$ are the conversion rates for three versions of the site. We would choose site A with the following probability  

$$ \frac{exp(p_A/ \tau)}{exp(p_A/ \tau) + exp(p_B/ \tau) + exp(p_C/ \tau)} $$

* $\tau$ is a choosen parameter that controls the ‘randomness’ of the
choice, usually around 0.001  


* It doesn’t take into account how many times it’s pulled an arm
* A member of probability matching algorithms, so-called for their property of creating the probability distribution

### Upper Confidence Bound (UCB)

* This is a class of algorithms that can be proven to have a logarithmic upper bound for regret. We won't worry about the mathematical proof and will just be focusing on the UCB1 algorithm  

* We choose the strategy that where the following value is the largest  

$$ p_A + \sqrt{\frac{2 log(N)}{n_A}} $$  

* where $N$ is the total number of rounds (for all sites) and $n_A$ is the number of times site A has been shown. $p_A$ is the conversion rate for that version so far  

* You should first make sure to play each bandit once so none of the $n_A$'s are 0  


* UCB doesn’t use randomness at all. Unlike epsilon-greedy, it’s possible to know exactly how UCB will
behave in any given situation. This can make it easier to
reason about at times  

* UCB doesn’t have free parameters that you need to
configure before you can deploy it. This is a major
improvement if you’re interested in running in the wild,
because it means that you can start UCB without having
clear sense of how you expect the world to behave 

### Bayesian Bandit  

* We can use the Bayesian Beta-Binomial conjugate prior
techniques used to model the click-through rate as our base model for each of the bandits  
* This is another probability matching algorithm where we have a separate model for each of the bandits  


* Each bandit has an associated beta distribution with parameters:  

    * $\alpha = 1 + $ number of times bandit has won
    * $\beta = 1 + $ number of times bandit has lost  
  
* Sample from each bandit’s distribution and play the bandit with the
highest value  


* It will naturally converge on the bandit with the best payout  

**Exercise**: Assume that the click-through rates of versions A and B of a website are $p_A = 0.53$ and $p_B = 0.67$. Simulate running the multi-arm bandit using these strategies. Plot (1) the percentage of time the optimal bandit was chosen over time, (2) the total regret over time for each algorithm.  